In [92]:
import anndata
import pandas as pd
import numpy as np

# Purpose of script

Group the GTEx data by organism part stratificataion and take the average expression. 

Just like with the single cell. Except we don't have cell info here so we arn't grouping with that as well

In [93]:
adata = anndata.read_h5ad('../../../../../pipeline42/datasets/Gtex/GTEx.h5ad')

In [94]:
adata.obs

,SAMPID,SMTS,SMTSD
SAMPID,,,
GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0226-SM-5GZZ7,Adipose Tissue,Adipose - Subcutaneous
GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0426-SM-5EGHI,Muscle,Muscle - Skeletal
GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0526-SM-5EGHJ,Blood Vessel,Artery - Tibial
GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0626-SM-5N9CS,Blood Vessel,Artery - Coronary
GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-0726-SM-5GIEN,Heart,Heart - Atrial Appendage
...,...,...,...
GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2126-SM-5EGIU,Ovary,Ovary
GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2226-SM-5EGIV,Vagina,Vagina
GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2426-SM-5E44I,Blood Vessel,Artery - Tibial


In [99]:
SMTS = adata.obs['SMTS'].unique()

In [100]:
example = adata[adata.obs['SMTS'] == SMTS[0]]

In [101]:
example.X

ArrayView([[0.000e+00, 8.764e+00, 0.000e+00, ..., 4.592e+00, 1.674e+01,
            2.430e+01],
           [0.000e+00, 5.389e+00, 0.000e+00, ..., 1.197e+00, 9.377e+00,
            8.912e+00],
           [0.000e+00, 8.380e-01, 0.000e+00, ..., 3.229e+00, 1.517e+01,
            5.419e+00],
           ...,
           [0.000e+00, 4.201e+00, 0.000e+00, ..., 3.657e+00, 1.112e+01,
            0.000e+00],
           [0.000e+00, 1.454e+00, 0.000e+00, ..., 3.249e+00, 3.778e+01,
            0.000e+00],
           [2.522e-02, 2.167e+00, 0.000e+00, ..., 3.357e+00, 1.395e+01,
            0.000e+00]], dtype=float32)

In [29]:
genewise_average = np.mean(example.X, axis = 0)

In [91]:
adata.X

array([[0.000e+00, 8.764e+00, 0.000e+00, ..., 4.592e+00, 1.674e+01,
        2.430e+01],
       [0.000e+00, 3.861e+00, 0.000e+00, ..., 2.339e+00, 2.824e+01,
        6.247e+00],
       [0.000e+00, 7.349e+00, 1.004e+00, ..., 3.886e+00, 1.517e+01,
        2.511e+01],
       ...,
       [0.000e+00, 2.857e+00, 0.000e+00, ..., 3.151e+00, 1.081e+01,
              nan],
       [1.965e-02, 8.696e-01, 0.000e+00, ..., 1.662e+00, 3.447e+01,
              nan],
       [2.522e-02, 2.167e+00, 0.000e+00, ..., 3.357e+00, 1.395e+01,
              nan]], dtype=float32)

In [112]:

def create_OP_list(adata, grouping_col:str):
    """
    Takes in adata object from bulk data and retusn a list of organism parts

    grouping_col is the column name containing the Organism Part Information
    """
    return adata.obs[grouping_col].unique()

OPs = create_OP_list(adata, grouping_col='SMTS')




def get_genewise_averages(adata, grouping_col:str, OPs:list):
    """
    grouping_col:str is a string indicating the column name of the column you want to subset by. Example for my data is that I wnat to group by organism parts. so I will use SMTS

    OPs: List of organism parts (str)

    Purpose: Function takes adata with a column of organism parts. Calculates the average exprsesion for each gene in each organism part. And returns a matrix containing these averages
    """


    group_averages = []

    for group in OPs:
        print(f'working on {group}')

        # Subset adata by the specific group of interest
        adata_group = adata[adata.obs[grouping_col] == group]

        genewise_average = np.mean(adata_group.X, axis = 0)
        
        group_averages.append(genewise_average)

    return np.matrix(group_averages)

genewise_averages = get_genewise_averages(adata=adata, grouping_col='SMTS', OPs = OPs)




working on Adipose Tissue
working on Muscle
working on Blood Vessel
working on Heart
working on Uterus
working on Vagina
working on Breast
working on Skin
working on Salivary Gland
working on Brain
working on Adrenal Gland
working on Thyroid
working on Lung
working on Spleen
working on Pancreas
working on Esophagus
working on Stomach
working on Colon
working on Small Intestine
working on Prostate
working on Testis
working on Nerve
working on Blood
working on Pituitary
working on Ovary
working on Liver
working on Kidney
working on Cervix Uteri
working on Fallopian Tube
working on Bladder


In [103]:
genewise_averages[:,-1]

matrix([[10.832185 ],
        [ 3.7126234],
        [16.562885 ],
        [ 4.831588 ],
        [27.259022 ],
        [13.491089 ],
        [13.653842 ],
        [ 9.780465 ],
        [12.5273695],
        [16.89485  ],
        [10.6826315],
        [28.530851 ],
        [11.648892 ],
        [ 5.944318 ],
        [ 4.729068 ],
        [14.292527 ],
        [ 9.621394 ],
        [15.904528 ],
        [ 8.064366 ],
        [18.114244 ],
        [15.765406 ],
        [19.887255 ],
        [ 1.4188132],
        [30.214485 ],
        [29.435663 ],
        [ 3.0021825],
        [11.901696 ],
        [ 0.       ],
        [ 0.       ],
        [ 0.       ]], dtype=float32)

In [104]:
np.sum(np.isnan(genewise_averages), axis = 0)

matrix([[0, 0, 0, ..., 0, 0, 0]])

All NANs have been removed from the last row. Seemed to be a problem in the data

# Create PD dataframe of genewise_averages and save as a csv

In [115]:
bulk_grouped =pd.DataFrame(genewise_averages, columns=adata.var_names, index=OPs)

In [119]:
round(bulk_grouped,5)

Description,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,FAM138A,OR4G4P,OR4G11P,OR4F5,RP11-34P13.7,CICP27,...,RNU1-84P,RP11-867G23.10,NPAS4,SNORA43-3,MRPL11,CTD-3074O7.7,PELI3,CTD-3074O7.5,DPP3,BBS1
Adipose Tissue,0.00648,4.14990,0.01873,0.01743,0.00689,0.03939,0.05506,0.07151,0.03495,0.04231,...,0.00988,1.167850,0.462180,0.00955,22.507179,0.05198,10.040110,2.72016,15.335720,10.832180
Muscle,0.00835,1.66335,0.00907,0.02190,0.00924,0.05379,0.07922,0.11183,0.02323,0.00984,...,0.00767,0.106150,0.064490,0.00320,20.161039,0.01996,11.936100,1.44118,16.344250,3.712620
Blood Vessel,0.00678,4.18538,0.01245,0.01490,0.00697,0.03069,0.04529,0.05570,0.02883,0.02969,...,0.01419,6.504620,2.329160,0.00851,27.960180,0.12210,20.370911,2.90099,16.448160,16.562880
Heart,0.00784,1.41403,0.01363,0.02632,0.01100,0.06923,0.09609,0.17153,0.02409,0.01831,...,0.00207,2.061150,0.188240,0.00375,14.523070,0.01904,4.841730,0.81894,6.171710,4.831590
Uterus,0.01315,8.30789,0.03489,0.01640,0.00803,0.02896,0.03813,0.04167,0.04476,0.05469,...,0.02087,24.271099,9.988500,0.01713,32.256481,0.26804,26.929300,3.39448,16.724979,27.259020
Vagina,0.01177,6.10995,0.02336,0.01395,0.00751,0.03402,0.04401,0.05393,0.04716,0.10709,...,0.02296,4.047710,1.653960,0.00623,25.591980,0.20815,14.894300,2.19431,19.508200,13.491090
Breast,0.00725,4.84691,0.02182,0.01854,0.00712,0.04142,0.05368,0.06762,0.04292,0.04901,...,0.00767,1.354480,0.457950,0.00650,24.088840,0.07459,11.587770,2.92380,16.015610,13.653840
Skin,0.00770,5.27628,0.02695,0.02005,0.00708,0.03783,0.05055,0.06761,0.02979,0.02789,...,0.00997,0.642360,0.293810,0.01005,26.763830,0.04927,9.215470,2.11230,22.728580,9.780460
Salivary Gland,0.00572,3.89845,0.01258,0.01951,0.00605,0.02345,0.04597,0.04466,0.03272,0.06278,...,0.01112,0.555590,0.250620,0.01290,18.031040,0.08326,10.787050,2.48291,22.802999,12.527370
Brain,0.01074,2.83498,0.00993,0.03433,0.01086,0.07000,0.09541,0.14387,0.03865,0.03087,...,0.01028,1.448560,15.104800,0.02052,10.661440,0.12571,25.458731,2.54633,7.995250,16.894850


In [118]:
bulk_grouped.to_csv('../data/bulk/bulk_grouped.csv')